In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen, urlretrieve, quote
from urllib.parse import urljoin

url = 'http://www.gov.scot/publications/coronavirus-covid-19-trends-in-daily-data/'
u = urlopen(url)
try:
    html = u.read().decode('utf-8')
finally:
    u.close()

soup = BeautifulSoup(html, "html.parser")

files = set()
for link in soup.select('a[href^=""]'):
    href = link.get('href')

    if not any(href.endswith(x) for x in ['.csv','.xls','.xlsx']):
        continue

    filename = href.rsplit('/', 1)[-1]
    ref = 'http://www.gov.scot/' + href
    files.add(ref)

In [52]:
for i in files:
    if 'nhs-board' in i:
        daily_figures = pd.read_excel(i, sheet_name = 'Table 1 - Cumulative cases', header=2)

In [53]:
daily_figures.iloc[:,1:] = daily_figures.iloc[:,1:].apply(lambda x:pd.to_numeric(x, errors='coerce'))
daily_figures_diff = daily_figures.diff()
daily_figures_diff['Date'] = daily_figures['Date notified']

In [ ]:
daily_figures_diff.tail()

In [56]:
import plotly.express as px
fig = px.line(daily_figures_diff, x='Date', y='NHS Lothian')
fig.show()